In [1]:
# Building Predictive Models
import pandas as pd
import os
import numpy as np

In [2]:
#import data
raw_data_path = os.path.join(os.path.pardir, 'data', 'processed')
train_file_path = os.path.join(raw_data_path, 'train.csv')
test_file_path = os.path.join(raw_data_path, 'test.csv')
train_df = pd.read_csv(train_file_path, index_col='PassengerId')
test_df = pd.read_csv(test_file_path, index_col='PassengerId')

In [3]:
 train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 34 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Survived            891 non-null    int64  
 1   Age                 891 non-null    float64
 2   Fare                891 non-null    float64
 3   FamilySize          891 non-null    int64  
 4   IsMother            891 non-null    int64  
 5   IsFemale            891 non-null    int64  
 6   Deck_A              891 non-null    int64  
 7   Deck_B              891 non-null    int64  
 8   Deck_C              891 non-null    int64  
 9   Deck_D              891 non-null    int64  
 10  Deck_E              891 non-null    int64  
 11  Deck_F              891 non-null    int64  
 12  Deck_G              891 non-null    int64  
 13  Deck_T              891 non-null    int64  
 14  Deck_Z              891 non-null    int64  
 15  Pclass_1            891 non-null    int64  
 16  Pclass_2

In [4]:
# Data Preparation
# Upper case for matrix, lower for vector
X = train_df.loc[:,'Age':].to_numpy().astype('float')
y = train_df['Survived'].ravel()

In [5]:
print(X.shape, y.shape)

(891, 33) (891,)


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape, y_train.shape)
print(X_test.shape,y_test.shape)

(712, 33) (712,)
(179, 33) (179,)


In [7]:
print('mean survival in train: {0:.3f}'.format(np.mean(y_train)))
print('mean survival in test: {0:.3f}'.format(np.mean(y_test)))

mean survival in train: 0.383
mean survival in test: 0.385


In [8]:
import sklearn
sklearn.__version__

'0.24.1'

In [9]:
# Baseline Model
from sklearn.dummy import DummyClassifier

#create model
model_dummy = DummyClassifier(strategy='most_frequent', random_state=0)

In [10]:
#train model
model_dummy.fit(X_train, y_train)

DummyClassifier(random_state=0, strategy='most_frequent')

In [11]:
print('score for baseline model : {0:.2f}'.format(model_dummy.score(X_test, y_test))) #default score is accuracy,

score for baseline model : 0.61


In [12]:
# performance metrics
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

# accuracy score
print('accuracy for baseline model : {0:.2f}'.format(accuracy_score(y_test, model_dummy.predict(X_test))))

accuracy for baseline model : 0.61


In [13]:
# confusion matric
print('confusion matrix for baseline model: \n {0}'.format(confusion_matrix(y_test, model_dummy.predict(X_test))))

confusion matrix for baseline model: 
 [[110   0]
 [ 69   0]]


In [14]:
# precision and recall scores
print('precision for baseline model : {0:.2f}'.format(precision_score(y_test, model_dummy.predict(X_test))))
print('recall for baseline model : {0:.2f}'.format(recall_score(y_test, model_dummy.predict(X_test))))

precision for baseline model : 0.00
recall for baseline model : 0.00


/Users/cherry/.pyenv/versions/ds37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
# Submission File

# converting to the matrix
test_X = test_df.to_numpy().astype('float')

In [16]:
# get predictions
predictions = model_dummy.predict(test_X)

In [17]:
df_submission = pd.DataFrame({'PassengerId': test_df.index, 'Survived': predictions})
df_submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [18]:
submission_data_path = os.path.join(os.path.pardir,'data', 'external')
submission_file_path = os.path.join(submission_data_path,'01_dummy.csv')

In [19]:
df_submission.to_csv(submission_file_path, index=False)

In [20]:
def get_submission_file(model, filename):
    # converting to the matrix
    test_X = test_df.to_numpy().astype('float')
    # get predictions
    predictions = model_dummy.predict(test_X)
    df_submission = pd.DataFrame({'PassengerId': test_df.index, 'Survived': predictions})
    df_submission.head()
    submission_data_path = os.path.join(os.path.pardir,'data', 'external')
    submission_file_path = os.path.join(submission_data_path,'01_dummy.csv')
    df_submission.to_csv(submission_file_path, index=False)

get_submission_file(model_dummy, '01_dummy.csv')

In [21]:
# Logistic Regression Model
from sklearn.linear_model import LogisticRegression

# create model
model_lr_1 = LogisticRegression(random_state=0, max_iter=1000)

#train model
model_lr_1.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=0)

In [22]:
print('score for logistic regression - version 1: {0:.2f}'.format(model_lr_1.score(X_test, y_test)))

score for logistic regression - version 1: 0.83


In [42]:
# Hyperparameter Optimization
# base model
model_lr = LogisticRegression(random_state=0, max_iter=2000, solver='liblinear')

from sklearn.model_selection import GridSearchCV
parameters = {'C': [1.0, 10.0, 50.0, 100.0, 1000.0], 'penalty' : ['l1', 'l2']}
clf = GridSearchCV(model_lr, param_grid=parameters, cv=3)

In [43]:
clf.fit(X_train,y_train)

GridSearchCV(cv=3,
             estimator=LogisticRegression(max_iter=2000, random_state=0,
                                          solver='liblinear'),
             param_grid={'C': [1.0, 10.0, 50.0, 100.0, 1000.0],
                         'penalty': ['l1', 'l2']})

In [44]:
clf.best_params_

{'C': 1.0, 'penalty': 'l1'}

In [45]:
clf.best_score_

0.8286647992530346

In [47]:
# evaluate model
# score for logistic regression v2
clf.score(X_test, y_test)

0.8324022346368715